In [1]:
import pickle
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from pandarallel import pandarallel
from tqdm.auto import tqdm

import torch
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
    BertTokenizerFast,
)

from dataset import BERTDataset
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
    load_model,
    save_checkpoint,
    set_lr_scheduler,
)

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=4)

In [2]:
LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}

In [3]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages,)


Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


In [4]:
class AicupTopkEvidenceBERT_Test_Dataset(torch.utils.data.Dataset):
    """AicupTopkEvidenceBERTDataset class for AICUP dataset with top-k evidence sentences."""

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: AutoTokenizer,
        max_length: int = 32,
        topk: int = 5,
        # TrainMode : bool=True
    ):
        self.data=data
        self.tokenizer=tokenizer
        self.max_length=max_length
    def __len__(self):
        return len(self.data)
    def __getitem__(
        self,
        idx: int,
        **kwargs,
    ) :
        item = self.data.iloc[idx]
        sentA = item["claim"]
        predicted_pages_text={}
        sentB = item["evidence_list"]
        
        for num,text in enumerate(sentB):
            PredictedTextToken = self.tokenizer(
            sentA,text,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
            )
            CandidateNum=f"ClaimEvidence{num+1}"
            predicted_pages_text[CandidateNum]=PredictedTextToken
        
        
        for key,value in predicted_pages_text.items():
            predicted_pages_text[key]={k: torch.tensor(v) for k, v in predicted_pages_text[key].items()}
        
        return predicted_pages_text

In [5]:
def RuleBasedAccuracy(y_pred,y_pred2,y_pred3,y_pred4,y_pred5):
    final_pred=[]
    
    for num,i in enumerate (y_pred):
        if(i==0 or y_pred2[num]==0 or y_pred3[num]==0 or y_pred4[num]==0 or y_pred5[num]==0):
            final_pred.append(0)
        elif(i==1 or y_pred2[num]==1 or y_pred3[num]==1 or y_pred4[num]==1 or y_pred5[num]==1):
            final_pred.append(1)
        else:
            final_pred.append(2)
    
    return final_pred
        
    


In [6]:
def RuleBasedAccuracy_1(y_pred):
    final_pred=[]
    
    for num,i in enumerate (y_pred):
        if(i==0  ):
            final_pred.append(0)
        elif(i==1 ):
            final_pred.append(1)
        else:
            final_pred.append(2)
    
    return final_pred
def RuleBasedAccuracy_2(y_pred,y_pred2):
    final_pred=[]
    
    for num,i in enumerate (y_pred):
        if(i==0 or y_pred2[num]==0 ):
            final_pred.append(0)
        elif(i==1 or y_pred2[num]==1 ):
            final_pred.append(1)
        else:
            final_pred.append(2)
    
    return final_pred
        
def RuleBasedAccuracy_3(y_pred,y_pred2,y_pred3):
    final_pred=[]
    
    for num,i in enumerate (y_pred):
        if(i==0 or y_pred2[num]==0 or y_pred3[num]==0):
            final_pred.append(0)
        elif(i==1 or y_pred2[num]==1 or y_pred2[num]==1):
            final_pred.append(1)
        else:
            final_pred.append(2)
    
    return final_pred

def RuleBasedAccuracy_4(y_pred,y_pred2,y_pred3,y_pred4):
    final_pred=[]
    
    for num,i in enumerate (y_pred):
        if(i==0 or y_pred2[num]==0 or y_pred3[num]==0 or y_pred4[num]==0 ):
            final_pred.append(0)
        elif(i==1 or y_pred2[num]==1 or y_pred3[num]==1 or y_pred4[num]==1 ):
            final_pred.append(1)
        else:
            final_pred.append(2)
    
    return final_pred
        
        
    



In [7]:
def run_predict(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    y_true = []
    y_pred = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    
    count=0
    Prediction={}
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
        

        ####### ClaimEvidence1 #########
        batch_2=batch["ClaimEvidence1"]
            #batch_2["labels"]=batch["labels"]
        batch_2 = {k: v.to(device) for k, v in batch_2.items()}
            
        outputs = model(**batch_2)
        ####### ClaimEvidence2 #########
        batch_2_2=batch["ClaimEvidence2"]
        batch_2_2 = {k: v.to(device) for k, v in batch_2_2.items()}
        outputs2 = model(**batch_2_2)
            
        ####### ClaimEvidence3 #########
        batch_2_3=batch["ClaimEvidence3"]
        batch_2_3 = {k: v.to(device) for k, v in batch_2_3.items()}
        outputs3 = model(**batch_2_3)
            
        ####### ClaimEvidence4 #########
        batch_2_4=batch["ClaimEvidence4"]
        batch_2_4 = {k: v.to(device) for k, v in batch_2_4.items()}
        outputs4 = model(**batch_2_4)

        ####### ClaimEvidence5 #########
        batch_2_5=batch["ClaimEvidence5"]
        batch_2_5 = {k: v.to(device) for k, v in batch_2_5.items()}
        outputs5 = model(**batch_2_5)



        
        logits = outputs.logits#logits shape:torch.Size([32, 3])
        logits2 = outputs2.logits
        logits3 = outputs3.logits
        logits4 = outputs4.logits
        logits5 = outputs5.logits


        y_pred.extend(torch.argmax(logits, dim=1).tolist())
        y_pred2.extend(torch.argmax(logits2, dim=1).tolist())
        y_pred3.extend(torch.argmax(logits3, dim=1).tolist())
        y_pred4.extend(torch.argmax(logits4, dim=1).tolist())
        y_pred5.extend(torch.argmax(logits5, dim=1).tolist())
        

            
    preds=RuleBasedAccuracy(y_pred,y_pred2,y_pred3,y_pred4,y_pred5)


    return preds

In [8]:
def run_predict_1(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    y_true = []
    y_pred = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    
    count=0
    Prediction={}
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
        
        ####### ClaimEvidence1 #########
        batch_2=batch["ClaimEvidence1"]
            #batch_2["labels"]=batch["labels"]
        batch_2 = {k: v.to(device) for k, v in batch_2.items()}
            
        outputs = model(**batch_2)
        
        logits = outputs.logits#logits shape:torch.Size([32, 3])
      


        y_pred.extend(torch.argmax(logits, dim=1).tolist())
        

            
    preds=RuleBasedAccuracy_1(y_pred)


    return preds

In [9]:
def run_predict_2(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    y_true = []
    y_pred = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    
    count=0
    Prediction={}
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
        
        ####### ClaimEvidence1 #########
        batch_2=batch["ClaimEvidence1"]
            #batch_2["labels"]=batch["labels"]
        batch_2 = {k: v.to(device) for k, v in batch_2.items()}
            
        outputs = model(**batch_2)
        ####### ClaimEvidence2 #########
        batch_2_2=batch["ClaimEvidence2"]
        batch_2_2 = {k: v.to(device) for k, v in batch_2_2.items()}
        outputs2 = model(**batch_2_2)
            
       
        logits = outputs.logits#logits shape:torch.Size([32, 3])
        logits2 = outputs2.logits
        


        y_pred.extend(torch.argmax(logits, dim=1).tolist())
        y_pred2.extend(torch.argmax(logits2, dim=1).tolist())
        

            
    preds=RuleBasedAccuracy_2(y_pred,y_pred2)


    return preds

In [10]:
def run_predict_3(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    y_true = []
    y_pred = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    
    count=0
    Prediction={}
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
       

        ####### ClaimEvidence1 #########
        batch_2=batch["ClaimEvidence1"]
            #batch_2["labels"]=batch["labels"]
        batch_2 = {k: v.to(device) for k, v in batch_2.items()}
            
        outputs = model(**batch_2)
        ####### ClaimEvidence2 #########
        batch_2_2=batch["ClaimEvidence2"]
        batch_2_2 = {k: v.to(device) for k, v in batch_2_2.items()}
        outputs2 = model(**batch_2_2)
            
        ####### ClaimEvidence3 #########
        batch_2_3=batch["ClaimEvidence3"]
        batch_2_3 = {k: v.to(device) for k, v in batch_2_3.items()}
        outputs3 = model(**batch_2_3)
            
        
        logits = outputs.logits#logits shape:torch.Size([32, 3])
        logits2 = outputs2.logits
        logits3 = outputs3.logits
        


        y_pred.extend(torch.argmax(logits, dim=1).tolist())
        y_pred2.extend(torch.argmax(logits2, dim=1).tolist())
        y_pred3.extend(torch.argmax(logits3, dim=1).tolist())
        

            
    preds=RuleBasedAccuracy_3(y_pred,y_pred2,y_pred3)


    return preds

In [11]:
def run_predict_4(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    y_true = []
    y_pred = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    
    count=0
    Prediction={}
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
       

        ####### ClaimEvidence1 #########
        batch_2=batch["ClaimEvidence1"]
            #batch_2["labels"]=batch["labels"]
        batch_2 = {k: v.to(device) for k, v in batch_2.items()}
            
        outputs = model(**batch_2)
        ####### ClaimEvidence2 #########
        batch_2_2=batch["ClaimEvidence2"]
        batch_2_2 = {k: v.to(device) for k, v in batch_2_2.items()}
        outputs2 = model(**batch_2_2)
            
        ####### ClaimEvidence3 #########
        batch_2_3=batch["ClaimEvidence3"]
        batch_2_3 = {k: v.to(device) for k, v in batch_2_3.items()}
        outputs3 = model(**batch_2_3)
            
        ####### ClaimEvidence4 #########
        batch_2_4=batch["ClaimEvidence4"]
        batch_2_4 = {k: v.to(device) for k, v in batch_2_4.items()}
        outputs4 = model(**batch_2_4)

       
        logits = outputs.logits#logits shape:torch.Size([32, 3])
        logits2 = outputs2.logits
        logits3 = outputs3.logits
        logits4 = outputs4.logits
        # logits5 = outputs5.logits


        y_pred.extend(torch.argmax(logits, dim=1).tolist())
        y_pred2.extend(torch.argmax(logits2, dim=1).tolist())
        y_pred3.extend(torch.argmax(logits3, dim=1).tolist())
        y_pred4.extend(torch.argmax(logits4, dim=1).tolist())
        

            
    preds=RuleBasedAccuracy_4(y_pred,y_pred2,y_pred3,y_pred4)


    return preds

In [12]:
def join_with_topk_evidence(
    df: pd.DataFrame,
    mapping: dict,
    mode: str = "train",
    topk: int = 5,
    train_evidence_split=True
) -> pd.DataFrame:
    """join_with_topk_evidence join the dataset with topk evidence.

    Note:
        After extraction, the dataset will be like this:
               id     label         claim                           evidence            evidence_list
        0    4604  supports       高行健...     [[[3393, 3552, 高行健, 0], [...  [高行健 （ ）江西赣州出...
        ..    ...       ...            ...                                ...                     ...
        945  2095  supports       美國總...  [[[1879, 2032, 吉米·卡特, 16], [...  [卸任后 ， 卡特積極參與...
        停各种战争及人質危機的斡旋工作 ， 反对美国小布什政府攻打伊拉克...

        [946 rows x 5 columns]

    Args:
        df (pd.DataFrame): The dataset with evidence.
        wiki_pages (pd.DataFrame): The wiki pages dataframe
        topk (int, optional): The topk evidence. Defaults to 5.
        cache(Union[Path, str], optional): The cache file path. Defaults to None.
            If cache is None, return the result directly.

    Returns:
        pd.DataFrame: The dataset with topk evidence_list.
            The `evidence_list` column will be: List[str]
    """

    # format evidence column to List[List[Tuple[str, str, str, str]]]
    if "evidence" in df.columns:
        df["evidence"] = df["evidence"].parallel_map(
            lambda x: [[x]] if not isinstance(x[0], list) else [x]
            if not isinstance(x[0][0], list) else x)

    print(f"Extracting evidence_list for the {mode} mode ...")
    if mode == "eval":
        # extract evidence
        df["evidence_list"] = df["predicted_evidence"].parallel_map(lambda x: [
            mapping.get(evi_id, {}).get(str(evi_idx), "")
            for evi_id, evi_idx in x  # for each evidence list
        ][:topk] if isinstance(x, list) else [])
        print(df["evidence_list"][:5])
    else:
        
        # extract evidence
        df["evidence_list"] = df["evidence"].parallel_map(lambda x: [
            " ".join([  # join evidence
                mapping.get(evi_id, {}).get(str(evi_idx), "")
                for _, _, evi_id, evi_idx in evi_list
            ]) if isinstance(evi_list, list) else ""
            for evi_list in x  # for each evidence list
            ][:len(x)] if isinstance(x, list) else [])

    return df

In [13]:
#@title  { display-mode: "form" }

MODEL_NAME = "ckiplab/bert-base-chinese"
TEST_BATCH_SIZE = 32  #@param {type:"integer"}
MAX_SEQ_LEN = 256  #@param {type:"integer"}
EVIDENCE_TOPK = 5

In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL2ID),
)
model.to(device)

Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-c

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [15]:
TEST_DATA = load_json("data/Stage2/Private/Private_doc10sent5_BM25_ver1_final.jsonl")
TEST_PKL_FILE = Path("data/Stage2/Private/Private_doc10sent5_BM25_ver1_final.pkl")
print(f"Private Data:{len(TEST_DATA)}筆")

Private Data:8049筆


In [16]:
TEST_DATA_Frame=pd.DataFrame(TEST_DATA)
TEST_DATA_None=TEST_DATA_Frame[TEST_DATA_Frame["predicted_evidence"].map(len)==0]
TEST_DATA_Short=TEST_DATA_Frame[TEST_DATA_Frame["predicted_evidence"].map(len)<5]
TEST_DATA_Short=TEST_DATA_Short.drop(TEST_DATA_Short[TEST_DATA_Short["predicted_evidence"].map(len)==0].index)
TEST_DATA_Frame=TEST_DATA_Frame.drop(TEST_DATA_Frame[TEST_DATA_Frame["predicted_evidence"].map(len)<5].index)

In [17]:
if not TEST_PKL_FILE.exists():
    test_df = join_with_topk_evidence(
        TEST_DATA_Frame,
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    test_df.to_pickle(TEST_PKL_FILE, protocol=4)
else:
    with open(TEST_PKL_FILE, "rb") as f:
        test_df = pickle.load(f)


Extracting evidence_list for the eval mode ...


0    [這一目中的鳥有些體態雄健優美 ， 色彩豔麗 ， 其中不少是珍稀物種和經濟物種 ， 與人類的...
1    [教會剛成立時爲解決教會里發生的一些問題 ， 使徒 （ 宗徒 ） 們寫下許多書信 ， 這些書...
2    [1973年3月17日羅伯 · 昆蘭生於明尼蘇達州聖保羅市 。, 1973年3月17日羅伯 ...
3    [小威廉絲是上屆的女子單打冠軍 ， 但她在決賽敗給加比涅 · 穆古魯薩 。, 小威廉絲是上屆...
4    [劉秀爲太祖劉邦的九世孫 ， 景皇帝劉啓的七世孫 ， 長沙定王劉發之後 ， 出身於南陽郡的地...
Name: evidence_list, dtype: object


In [18]:
test_short_df = join_with_topk_evidence(
        TEST_DATA_Short,
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )

Extracting evidence_list for the eval mode ...


425    [成立於1982年 ， 原爲榮民總醫院臺中分院 ， 後於1988年獨立爲臺中榮民總醫院 。,...
441    [《 啓示錄 》 ， 天主教稱 《 若望默示錄 》 （ Αποκάλυψη του Ιωάν...
461    [標準的殉爆度測試方法 ， 是取兩段相同的炸藥 ， 分開一個距離 ， 引爆其中一段 ， 看另...
552    [穴棲蛇胸鱔爲輻鰭魚綱合鰓目合鰓魚科的其中一種 ， 被IUCN列爲次級保育動物 ， 分佈於中...
634    [永生龍 （ 屬名 ： Umoonasaurus ） 是蛇頸龍亞目的一屬 ， 屬於長鎖龍科 ...
Name: evidence_list, dtype: object


In [19]:
len(test_short_df)

56

In [20]:
OUTPUT_FILENAME = "submission.jsonl"

In [21]:
test_short_df_1=test_short_df[test_short_df["predicted_evidence"].map(len)==1]
test_short_df_2=test_short_df[test_short_df["predicted_evidence"].map(len)==2]
test_short_df_3=test_short_df[test_short_df["predicted_evidence"].map(len)==3]
test_short_df_4=test_short_df[test_short_df["predicted_evidence"].map(len)==4]

In [22]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")#BertTokenizerFast
test_dataset = AicupTopkEvidenceBERT_Test_Dataset(
    test_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
    topk=EVIDENCE_TOPK
)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [24]:
Sub_CKPT_DIR="weights/Stage3"
sub_ckpt_name= "Stage3_model.pt"
model = load_model(model, sub_ckpt_name, Sub_CKPT_DIR)

In [25]:

predicted_label = run_predict(model, test_dataloader, device)

Predicting:   0%|          | 0/1998 [00:00<?, ?it/s]

In [26]:
predict_dataset = test_df.copy()
predict_dataset["predicted_label"] = list(map(ID2LABEL.get, predicted_label))#row:7990


In [27]:
test_dataset_1 = AicupTopkEvidenceBERT_Test_Dataset(
    test_short_df_1,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
    topk=EVIDENCE_TOPK
)
test_dataloader_1 = DataLoader(test_dataset_1, batch_size=2)
predicted_label_1 = run_predict_1(model, test_dataloader_1, device)#row:3
predict_dataset_1=test_short_df_1.copy()
predict_dataset_1["predicted_label"] = list(map(ID2LABEL.get, predicted_label_1))

Predicting:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
test_dataset_2 = AicupTopkEvidenceBERT_Test_Dataset(
    test_short_df_2,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
    topk=EVIDENCE_TOPK
)
test_dataloader_2 = DataLoader(test_dataset_2, batch_size=2)

In [29]:
predicted_label_2 = run_predict_2(model, test_dataloader_2, device)#row:4

Predicting:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
predict_dataset_2=test_short_df_2.copy()
predict_dataset_2["predicted_label"] = list(map(ID2LABEL.get, predicted_label_2))

In [31]:
test_dataset_3 = AicupTopkEvidenceBERT_Test_Dataset(
    test_short_df_3,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
    topk=EVIDENCE_TOPK
)
test_dataloader_3 = DataLoader(test_dataset_3, batch_size=5)
predicted_label_3 = run_predict_3(model, test_dataloader_3, device) # 22
predict_dataset_3=test_short_df_3.copy()
predict_dataset_3["predicted_label"] = list(map(ID2LABEL.get, predicted_label_3))


Predicting:   0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
len(predicted_label_3)

22

In [33]:
test_dataset_4 = AicupTopkEvidenceBERT_Test_Dataset(
    test_short_df_4,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
    topk=EVIDENCE_TOPK
)
test_dataloader_4 = DataLoader(test_dataset_4, batch_size=5)
predicted_label_4 = run_predict_4(model, test_dataloader_4, device) # 27
predict_dataset_4=test_short_df_4.copy()
predict_dataset_4["predicted_label"] = list(map(ID2LABEL.get, predicted_label_4))


Predicting:   0%|          | 0/6 [00:00<?, ?it/s]

In [34]:
len(predicted_label_4)

27

In [35]:
TEST_DATA_None["predicted_evidence"]=None
TEST_DATA_None["predicted_label"]="NOT ENOUGH INFO"
predict_dataset_5=TEST_DATA_None

In [36]:
final_dataset=pd.concat([predict_dataset,predict_dataset_1,predict_dataset_2,predict_dataset_3,predict_dataset_4,predict_dataset_5])
# final_dataset=pd.concat([predict_dataset,predict_dataset_5])

In [37]:
len(predict_dataset_5)

3

In [38]:
final_dataset

,id,claim,predicted_pages,predicted_evidence,evidence_list,predicted_label
0,21498,雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與...,"[鳥, 顏色, 物種, 人類, 雞形目, 關係, 雞]","[[雞形目, 3], [雞形目, 2], [雞形目, 0], [雞形目, 1], [鳥, 0]]",[這一目中的鳥有些體態雄健優美 ， 色彩豔麗 ， 其中不少是珍稀物種和經濟物種 ， 與人類的...,NOT ENOUGH INFO
1,13037,教會剛建立時為解決內部的一些問題，使徒們寫下許多便條，其中有八卷不是保羅寫的為大公書信。,"[書信, 內部, 問題, 大公書信, 保羅·彼特尼, 克里斯·保羅, 教會, 保羅·欣德米特...","[[新約書信, 5], [大公書信, 0], [保羅_(使徒), 3], [大公書信, 4]...",[教會剛成立時爲解決教會里發生的一些問題 ， 使徒 （ 宗徒 ） 們寫下許多書信 ， 這些書...,refutes
2,18652,羅伯·昆蘭於明尼蘇達州聖保羅市出生。,"[羅伯·昆蘭, 明尼蘇達州, 聖保羅市]","[[羅伯·昆蘭, 4], [羅伯·昆蘭, 4], [羅伯·昆蘭, 0], [羅伯·昆蘭, 0...","[1973年3月17日羅伯 · 昆蘭生於明尼蘇達州聖保羅市 。, 1973年3月17日羅伯 ...",supports
3,21378,2015年美國網球公開賽女子單打比賽裡小威廉絲是上一屆的冠軍。,"[2015年, 美國網球公開賽, 蜜雪兒·威廉絲, 2016年法國網球公開賽, 冠軍, 愛莉...","[[2016年法國網球公開賽, 8], [2016年法國網球公開賽, 8], [2015年美...","[小威廉絲是上屆的女子單打冠軍 ， 但她在決賽敗給加比涅 · 穆古魯薩 。, 小威廉絲是上屆...",supports
4,18624,南陽郡的地方豪族出身的漢光武帝劉秀為太祖劉邦的九世孫。,"[漢, 世孫, 太祖, 漢光武帝, 劉邦, 太祖_(祖靈信仰), 南陽郡]","[[漢光武帝, 4], [漢光武帝, 0], [劉邦, 0], [漢光武帝, 6], [南陽...",[劉秀爲太祖劉邦的九世孫 ， 景皇帝劉啓的七世孫 ， 長沙定王劉發之後 ， 出身於南陽郡的地...,supports
...,...,...,...,...,...,...
6332,17195,土衛十一的命名與泰坦神的無關。,"[土衛十一, 土衛十]","[[土衛十一, 0], [土衛十, 4], [土衛十, 0], [土衛十, 1]]",[土衛十一又稱爲 「 艾比米修斯 」 （ Epimetheus ） ， 是土星的一顆內側衛星...,NOT ENOUGH INFO
7222,8370,馬世莉的哥哥因爲心肌梗塞過世。,[馬世莉],"[[馬世莉, 5], [馬世莉, 0], [馬世莉, 4], [馬世莉, 1]]",[因爲身爲家中唯一男丁的哥哥早逝 ， 基於延續馬家香火的想法 ， 將二姐的兒子過繼給未婚的馬...,NOT ENOUGH INFO
757,16890,。,[],None,NaN,NOT ENOUGH INFO
4893,15287,。,[],None,NaN,NOT ENOUGH INFO


In [39]:
final_dataset[["id", "predicted_label", "predicted_evidence"]].to_json(
    OUTPUT_FILENAME,
    orient="records",
    lines=True,
    force_ascii=False,
)